# Import

In [ ]:
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Hardware and framework

In [ ]:
system_details = pd.DataFrame({
    'Title':       ['GPU', 'CPU', 'RAM', 'Windows', 'Python', 'Tensorflow'],
    'Information': ['NVIDIA GeForce RTX 2080Ti', 'AMD Ryzen 9 3950X 16-Core Processor',
                    '128 GB', 'Windows 10 Enterprise', '3.7', '2.4']
    })
system_details

# Pretexts

## Pretext model

In [ ]:
pretext_parameters = pd.DataFrame({
    'Task':      ['Gender, rotation, and subject', 'Inpainting',
                  'Gender, rotation, and subject', 'Inpainting', 'all', 'all', 'all',
                  'all'],
    'Parameter': ['Loss', 'Loss', 'Metric', 'Metric', 'Learning rate', 'Optimizer',
                  'Batch size', 'Epoch'],
    'Value':     ['categorical cross entropy', 'mse', 'Accuracy', 'PSNR',
                  '0.01 (Reduce LR On Plateau to 0.000001)', 'RMSprop', '16', '300']
    })
pretext_parameters

## Inpainting - image

In [ ]:
def PSNR(super_resolution, high_resolution):
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val = 1)[0]
    return psnr_value


def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


model = load_model('checkpoint_loss2', custom_objects = {'PSNR': PSNR, 'SSIMLoss': SSIMLoss})
folder = 'F:/poorya/datasets/keypoint J dataset/pretext dataset/test/'
i = 50
n = os.listdir(folder)
n.sort(key = len)

images = []
mask = []
rotation = []
gender = []
subject = []

train_img_1 = image.img_to_array(image.load_img(folder + n[i], color_mode = "grayscale", target_size = (384, 384)))
train_img_2 = image.img_to_array(image.load_img(folder + n[i], color_mode = "grayscale", target_size = (384, 384)))
train_img_1 /= 255.0
train_img_2 /= 255.0

mask.append(train_img_1[168:216, 168:216])
train_img_2[168:216, 168:216] = (0)
images.append(train_img_2)

rotation.append(to_categorical(int(n[i][-10]), num_classes = 4))
subject.append(to_categorical(int(n[i][-6:-4]) - 1, num_classes = 89))
gender.append(to_categorical(int(n[i][-8]), num_classes = 2))
y1_pred, y2_pred, y3_pred, y4_pred = model.predict(np.array(images))

plt.imshow(np.array(mask)[0, :, :, :])
plt.show()
plt.imshow(y4_pred[0, :, :, :])
plt.show()

plt.imshow(train_img_1)
plt.show()

train_img_2[168:216, 168:216] = y4_pred[0, :, :, :]
plt.imshow(train_img_2)
plt.show()

## Pretexts results - table

In [ ]:
sub = []
my_subjects = {}
counter = 0

files = os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/train_aug/')

for i in files:
    sub.append(i[-6:-4])

for i in range(95):
    if str(i).zfill(2) in sub:
        my_subjects[str(i).zfill(2)] = counter
        counter = counter + 1

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, folder, batch_size, my_subjects):
        self.folder = folder
        self.n = os.listdir(folder)
        self.batch_size = batch_size
        self.my_subjects = my_subjects
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.n) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.n[k] for k in indexes]
        x, y = self.__data_generation(list_IDs_temp)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.n))

    def __data_generation(self, list_IDs_temp):
        images = np.empty((self.batch_size, 384, 384, 1))
        masks = np.empty((self.batch_size, 48, 48, 1))
        rotation = np.empty((self.batch_size), dtype = int)
        subject = np.empty((self.batch_size), dtype = int)
        gender = np.empty((self.batch_size), dtype = int)

        for i, ID in enumerate(list_IDs_temp):

            train_img_1 = image.img_to_array(image.load_img(self.folder + ID, color_mode = "grayscale", target_size = (
                384, 384)))
            train_img_1 /= 255.0
            masks[i,] = train_img_1[168:216, 168:216]

            train_img_2 = image.img_to_array(image.load_img(self.folder + ID, color_mode = "grayscale", target_size = (
                384, 384)))
            train_img_2 /= 255.0
            train_img_2[168:216, 168:216] = 0
            images[i,] = train_img_2

            rotation[i] = int(ID[-10])
            subject[i] = int(self.my_subjects[ID[-6:-4]])
            gender[i] = int(ID[-8])

        return images, [to_categorical(gender, num_classes = 2), to_categorical(rotation, num_classes = 4),
                        to_categorical(subject, num_classes = 89), masks]


bs = 16
test_gen = DataGenerator('F:/poorya/datasets/keypoint J dataset/pretext dataset/test/', bs, my_subjects)
model.evaluate(test_gen, steps = len(os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/val/')) // bs)

In [ ]:
pretext_results = pd.DataFrame({
    'Task':  ['Gender', 'Rotation', 'Subject', 'Inpainting'],
    'Value': ['Accuracy = 98.96', 'Accuracy = 99.31', 'Accuracy = 67.01', 'PSNR = 21.90']
    })
pretext_results

## Pretext confusion matrices - cm 

In [ ]:
def image_generator(folder):
    n = os.listdir(folder)
    n.sort(key = len)

    images = []
    rotation = []
    subject = []
    gender = []

    for i in range(len(n)):
        train_img_2 = image.img_to_array(image.load_img(folder + n[i], color_mode = "grayscale", target_size = (
            384, 384)))
        train_img_2 /= 255.0
        train_img_2[168:216, 168:216] = 0
        images.append(train_img_2)

        rotation.append(to_categorical(int(n[i][-10]), num_classes = 4))
        subject.append(to_categorical(int(my_subjects[n[i][-6:-4]]), num_classes = 89))
        gender.append(to_categorical(int(n[i][-8]), num_classes = 2))

    return np.array(images), [np.array(gender), np.array(rotation), np.array(subject)]


x_test, y_true = image_generator('F:/poorya/datasets/keypoint J dataset/pretext dataset/test/')
predictions = model.predict(x_test, batch_size = 1, verbose = 1, steps = len(os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/test/')) // 1)

In [ ]:
def change(x):
    answer = np.zeros((np.shape(x)[0]))
    for i in range(np.shape(x)[0]):
        max_value = max(x[i, :])
        max_index = list(x[i, :]).index(max_value)
        answer[i] = max_index
    return answer.astype(int)


labels = [['Male', 'Female'],
          ['0 degrees', '90 degrees', '180 degrees', '270 degrees'],
          [str(k + 1) for k in range(89)]]

cm = confusion_matrix(change(y_true[0]), change(predictions[0]))
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = labels[0])
disp.plot(cmap = plt.cm.Blues)
plt.savefig('1.jpg')
plt.show()

cm = confusion_matrix(change(y_true[1]), change(predictions[1]))
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = labels[1])
disp.plot(cmap = plt.cm.Blues)
plt.savefig('2.jpg')
plt.show()

sub_pred = np.append(change(predictions[2]), [73, 78])
sub_true = np.append(change(y_true[2]), [73, 78])
fig, ax = plt.subplots(figsize = (20, 20))
plt.rcParams.update({'font.size': 8})
cm = confusion_matrix(sub_true, sub_pred)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = labels[2])
disp.plot(cmap = plt.cm.Blues, ax = ax)
plt.savefig('3.jpg')
plt.show()

# Face detection

## Face detection model

In [ ]:
pretext_parameters = pd.DataFrame({
    'Parameter': ['Loss', 'Metric', 'Learning rate', 'Optimizer', 'Batch size',
                  'Epoch'],
    'Value':     ['mse', 'IoU', '0.05 (Reduce LR On Plateau to 0.000001)', 'Adam',
                  '8', '300']
    })
pretext_parameters

## Detected faces - image

In [ ]:
ts_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/test_faces.csv')

In [ ]:
from time import time

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
n = ts_df['name'].tolist()
i = 99

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + n[i], color_mode = "grayscale", target_size = (
    384, 512))
train_img = image.img_to_array(train_igg)
train_img = train_img[:, 64:448]
train_img /= 255.0
img[0, :, :, :] = train_img
img = np.array(img)

faced = load_model('checkpoint_face')
lbl_c = faced.predict(img)

fig, ax = plt.subplots(figsize = (20, 10))
ax.imshow(train_igg, cmap = 'gray')
rect = patches.Rectangle((lbl_c[0][1] + 64, lbl_c[0][2]), (lbl_c[0][0] + 64 - lbl_c[0][1] - 64), (
                       lbl_c[0][3] - lbl_c[0][2]), linewidth = 4, edgecolor = 'r', facecolor = 'none')
ax.add_patch(rect)
plt.show()

In [ ]:
summ = 0

for i in range(100):
    start = time()
    lbl_c = faced.predict(img)
    end = time()

    summ = summ + (end - start)

print(summ / 100)

## IoU (comparison) - plot

In [ ]:
def image_generator(ts_df):
    n = ts_df.name.to_list()
    images = np.empty((len(n), 384, 384, 1))
    iou = np.zeros((len(n), 384, 384, 1), dtype = int)
    face = np.empty((len(n), 4))

    for i, ID in enumerate(n):
        train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale", target_size = (
            384, 512)))
        train_img = train_img[:, 64:448]
        train_img /= 255.0
        images[i,] = train_img

        face[i, 0] = int(ts_df.iloc[i]['Xmax']) // 2
        face[i, 1] = int(ts_df.iloc[i]['Xmin']) // 2
        face[i, 2] = int(ts_df.iloc[i]['Ymin']) // 2
        face[i, 3] = int(ts_df.iloc[i]['Ymax']) // 2

        iou[i, int(face[i, 2]):int(face[i, 3]), int(face[i, 1]):int(face[i, 0]), 0] = 1

    return images, iou


x_test, y_true = image_generator(ts_df)

In [ ]:
predictions = faced.predict(x_test, batch_size = 1, verbose = 1, steps = len(ts_df) // 1)

In [ ]:
iou_pred = np.zeros((len(predictions), 384, 384, 1), dtype = int)

for idx, pred in enumerate(predictions):
    iou_pred[idx, int(pred[2]):int(pred[3]), int(pred[1]):int(pred[0]), 0] = 1

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes = 2)
m.update_state(y_true, iou_pred)
mean_iou = m.result().numpy()

print('mean iou: ', mean_iou)

In [ ]:
each_iou = np.zeros((len(predictions)))

for i in range(len(predictions)):
    mymetric = tf.keras.metrics.MeanIoU(num_classes = 2)
    mymetric.update_state(y_true[i], iou_pred[i])
    each_iou[i] = mymetric.result().numpy()

In [ ]:
r1 = np.mean(each_iou)
print("\nMean: ", r1)

r2 = np.std(each_iou)
print("\nstd: ", r2)

In [ ]:
iou_range = np.empty((11))

for myidx, band in enumerate(np.arange(0, 1.1, 0.1)):
    counter = 0

    for mydata in each_iou:
        if mydata > band:
            counter = counter + 1

    iou_range[myidx] = counter / len(each_iou)

In [ ]:
adrs = 'C:/Users/DrBah/Desktop/keypoint images/03 - Results/Face detection/04 IoU (comparison) - plot/'
file = os.listdir(adrs)
file.remove('1.png')
file.remove('2.png')
file.remove('3.png')

plt.figure(figsize = (20, 15))

mycsv = myy
tmp = mycsv[0]
for idx, mmbr in enumerate(mycsv):
    if mmbr > tmp:
        mycsv[idx] = tmp

plt.plot(myx, mycsv, linewidth = 5, markersize = 12, label = 'our method')

markers = ['|', 'v', '*', 's', 'x', 'D', 'p']

for idxx, myfile in enumerate(file):
    csv = pd.read_csv(adrs + myfile, header = None)

    myx, myy = smoothing(csv[1].to_list())
    mycsv = myy
    plt.plot(myx, mycsv, linewidth = 5, markersize = 12, label = myfile[:-4])

plt.xlabel('Intersection Over Union (IoU)', fontsize = 25)
plt.ylabel('True positive (%)', fontsize = 25)
plt.rcParams.update({'font.size': 20})
plt.legend(prop = {'size': 20})
plt.axis([-0.01, 1, 0, 101])
plt.grid()
plt.show()

In [ ]:
adrs = 'F:/Poorya/datasets/labels/new-labels/'
up_lbl = pd.read_csv(adrs + 'output.csv')
up_lbl.columns = ['name']

files = os.listdir(adrs)
files.remove('output.csv')

In [ ]:
main_df = pd.read_csv('F:/Poorya/datasets/ThermalFaceDatabase/main_df.csv')


def df_to_ary(df):
    ary = np.zeros((len(df), 136), dtype = int)
    fth = np.zeros((len(df), 4), dtype = int)

    for i in range(len(df)):
        rep_x = [j for j in range(len(df['x'][i])) if df['x'][i].startswith(',', j)]
        rep_y = [j for j in range(len(df['y'][i])) if df['y'][i].startswith(',', j)]
        ary[i, 0] = int(df['x'][i][1:rep_x[0]]) / 2
        ary[i, 67] = int(df['x'][i][rep_x[-1] + 1:-1]) / 2
        ary[i, 68] = (int(df['y'][i][1:rep_y[0]]) - 128) / 2
        ary[i, 135] = (int(df['y'][i][rep_y[-1] + 1:-1]) - 128) / 2
        for k in range(66):
            ary[i, k + 1] = int(df['x'][i][rep_x[k] + 1:rep_x[k + 1]]) / 2
            ary[i, k + 69] = (int(df['y'][i][rep_y[k] + 1:rep_y[k + 1]]) - 128) / 2

        fth[i, 2] = min(ary[i, :68])  # y1
        fth[i, 3] = max(ary[i, :68])  # y2
        fth[i, 1] = min(ary[i, 68:])  # x1
        fth[i, 0] = max(ary[i, 68:])  # x2

    return fth


main_ary = df_to_ary(main_df)

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
i = 99

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + main_df.iloc[i][
    'images'], color_mode = "grayscale", target_size = (384, 512))
train_img = image.img_to_array(train_igg)
train_img = train_img[:, 64:448]
train_img /= 255.0
img[0, :, :, :] = train_img
img = np.array(img)

lbl_c = main_ary[i]

fig, ax = plt.subplots(figsize = (20, 10))
ax.imshow(train_igg, cmap = 'gray')
rect = patches.Rectangle((lbl_c[1] + 64, lbl_c[2]), (lbl_c[0] + 64 - lbl_c[1] - 64), (
                       lbl_c[3] - lbl_c[2]), linewidth = 4, edgecolor = 'r', facecolor = 'none')
ax.add_patch(rect)
plt.show()

In [ ]:
test_df = pd.DataFrame(columns = ['name', 'Xmax', 'Ymin', 'Xmin', 'Ymax'])

for idx, img in enumerate(up_lbl.iloc):
    sprt = img['name'].index(':')
    index = main_df.isin([img['name'][:sprt]]).any(axis = 1).idxmax()
    x1 = main_ary[index, 1]
    x2 = main_ary[index, 0]
    y2 = main_ary[index, 3]

    test_df.loc[idx] = {
        'name': img['name'][:sprt],
        'Xmax': x2,
        'Ymin': int(int(img['name'][sprt + 2:]) / 2),
        'Xmin': x1,
        'Ymax': y2
        }

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
test = test_df.iloc[10]

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + test[
    'name'], color_mode = "grayscale", target_size = (384, 512))
train_img = image.img_to_array(train_igg)
train_img = train_img[:, 64:448]
train_img /= 255.0
img[0, :, :, :] = train_img
img = np.array(img)

lbl_c = main_ary[i]

fig, ax = plt.subplots(figsize = (20, 10))
ax.imshow(train_igg, cmap = 'gray')
rect = patches.Rectangle((test['Xmin'] + 64, test['Ymin']), (test['Xmax'] + 64 - test['Xmin'] - 64), (
                       test['Ymax'] - test['Ymin']), linewidth = 4, edgecolor = 'r', facecolor = 'none')
ax.add_patch(rect)
plt.show()

In [ ]:
def image_generator(ts_df):
    n = ts_df.name.to_list()
    images = np.empty((len(n), 384, 384, 1))
    iou = np.zeros((len(n), 384, 384, 1), dtype = int)
    face = np.empty((len(n), 4))

    for i, ID in enumerate(n):
        train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale", target_size = (
            384, 512)))
        train_img = train_img[:, 64:448]
        train_img /= 255.0
        images[i,] = train_img

        face[i, 0] = int(ts_df.iloc[i]['Xmax'])
        face[i, 1] = int(ts_df.iloc[i]['Xmin'])
        face[i, 2] = int(ts_df.iloc[i]['Ymin'])
        face[i, 3] = int(ts_df.iloc[i]['Ymax'])

        iou[i, int(face[i, 2]):int(face[i, 3]), int(face[i, 1]):int(face[i, 0]), 0] = 1

    return images, iou


x_test, y_true = image_generator(test_df)

In [ ]:
plt.imshow(x_test[0])
plt.show()
plt.imshow(y_true[0])
plt.show()

In [ ]:
faced = load_model('checkpoint_face')
predictions = faced.predict(x_test, batch_size = 1, verbose = 1, steps = len(test_df) // 1)
iou_pred = np.zeros((len(predictions), 384, 384, 1), dtype = int)

for idx, pred in enumerate(predictions):
    iou_pred[idx, int(pred[2]):int(pred[3]), int(pred[1]):int(pred[0]), 0] = 1

m = tf.keras.metrics.MeanIoU(num_classes = 2)
m.update_state(y_true, iou_pred)
mean_iou = m.result().numpy()

print('mean iou: ', mean_iou)

In [ ]:
each_iou = np.zeros((len(predictions)))

for i in range(len(predictions)):
    mymetric = tf.keras.metrics.MeanIoU(num_classes = 2)
    mymetric.update_state(y_true[i], iou_pred[i])
    each_iou[i] = mymetric.result().numpy()

r1 = np.mean(each_iou)
print("\nMean: ", r1)

r2 = np.std(each_iou)
print("\nstd: ", r2)

In [ ]:
iou_range = np.empty((11))

for myidx, band in enumerate(np.arange(0, 1.1, 0.1)):
    counter = 0

    for mydata in each_iou:
        if mydata > band:
            counter = counter + 1

    iou_range[myidx] = counter / len(each_iou)

In [ ]:
adrs = 'C:/Users/DrBah/Desktop/keypoint images/03 - Results/Face detection/04 IoU (comparison) - plot/'
file = os.listdir(adrs)
file.remove('1.png')

plt.figure(figsize = (20, 15))
plt.plot(np.arange(0, 1.1, 0.1), iou_range * 100, marker = "o", linewidth = 3, markersize = 12, label = 'our method')

markers = ['|', 'v', '*', 's', 'x', 'D', 'p']

for idxx, myfile in enumerate(file):
    csv = pd.read_csv(adrs + myfile, header = None)
    plt.plot(np.arange(0, 1.1, 0.1),
             csv[1].to_list(), marker = markers[idxx], linewidth = 3, markersize = 12, label = myfile[:-4])

plt.xlabel('Intersection Over Union (IoU)', fontsize = 25)
plt.ylabel('True positive (%)', fontsize = 25)
plt.rcParams.update({'font.size': 50})
plt.legend(prop = {'size': 20})
plt.axis([-0.01, 1, 0, 101])
plt.show()

# Keypoint detection

## keypoint model

In [ ]:
pretext_parameters = pd.DataFrame({
    'Parameter': ['Loss', 'Metric', 'Learning rate', 'Optimizer', 'Batch size',
                  'Epoch'],
    'Value':     ['mse', 'Mean Absolute Percentage Error',
                  '0.001 (Reduce LR On Plateau to 0.000001)', 'RMSprop', '2', '50']
    })
pretext_parameters

## converge

# Different tasks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
materials = ['Subject', 'Gender', 'Inpainting', 'Rotation', 'Inpainting + Rotation', 'Gender + Inpainting + Rotation',
             'Our pretexts']
x_pos = np.arange(len(materials))

IoU_mean = [0.85, 0.89, 0.91, 0.92, 0.932, 0.939, 0.94]
NME_mean = [2.22, 1.54, 1.74, 2.68, 1.44, 2.13, 1.29]

In [ ]:
# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, IoU_mean, align = 'center', alpha = 0.5, ecolor = 'black', capsize = 10,
       color = ['pink', 'orange', 'green', 'red', 'purple', 'olive', 'blue'])
ax.set_ylabel('Intersection over Union (IoU)')
ax.set_xticks(x_pos)
ax.set_xticklabels(materials)
ax.yaxis.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Build the plot
fig, ax = plt.subplots()
ax.bar(x_pos, NME_mean, align = 'center', alpha = 0.5, ecolor = 'black', capsize = 10,
       color = ['pink', 'orange', 'green', 'red', 'purple', 'olive', 'blue'])
ax.set_ylabel('Normalized Mean Error (NME)')
ax.set_xticks(x_pos)
ax.set_xticklabels(materials)
ax.yaxis.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
adrs = 'C:/Users/DrBah/Desktop/keypoint images/03 - Results/Keypoint detection/05 NME for different pretexts - bar plot/'
file = os.listdir(adrs)
file.sort(key = len)

plt.figure(figsize = (20, 15))

for idxx, myfile in enumerate(file):
    csv = np.load(adrs + myfile)
    plt.plot(csv[0], csv[1], linewidth = 5, markersize = 12, label = myfile[:-4])

plt.rcParams.update({'font.size': 20})
plt.legend(prop = {'size': 20})
plt.axis([0, 0.2, 0, 1])
plt.grid()
plt.show()

In [ ]:
ts_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/test_kp.csv')


def df_to_ary(df):
    ary = np.zeros((len(df), 136), dtype = int)
    for i in range(len(df)):
        rep_x = [j for j in range(len(df['x'][i])) if df['x'][i].startswith(',', j)]
        rep_y = [j for j in range(len(df['y'][i])) if df['y'][i].startswith(',', j)]
        ary[i, 0] = int(df['x'][i][1:rep_x[0]]) / 2
        ary[i, 67] = int(df['x'][i][rep_x[-1] + 1:-1]) / 2
        ary[i, 68] = (int(df['y'][i][1:rep_y[0]]) - 128) / 2
        ary[i, 135] = (int(df['y'][i][rep_y[-1] + 1:-1]) - 128) / 2
        for k in range(66):
            ary[i, k + 1] = int(df['x'][i][rep_x[k] + 1:rep_x[k + 1]]) / 2
            ary[i, k + 69] = (int(df['y'][i][rep_y[k] + 1:rep_y[k + 1]]) - 128) / 2

    return ary


ts_ary = df_to_ary(ts_df)

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
adrs = 'F:/Poorya/keypoint detection J/final codes/Pretexts/00 NME/'
file = os.listdir(adrs)
file.remove('Label.npy')

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ts_df['images'][
    193], color_mode = "grayscale", target_size = (384, 512))
train_img = image.img_to_array(train_igg)
train_img /= 255.0
img[0, :, :, :] = train_img[:, 64:448]
img = np.array(img)

plt.figure(figsize = (20, 10))
plt.imshow(img[0], cmap = 'gray')

lbl = np.load(adrs + 'Label.npy')
plt.scatter(lbl[68:], lbl[0:68], label = 'Label')

for idx, myfile in enumerate(file):
    lbl = np.load(adrs + myfile)
    plt.scatter(lbl[0][68:], lbl[0][0:68], label = myfile[:-4])

plt.axis([100, 290, 220, 350])
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model, Model


def PSNR(super_resolution, high_resolution):
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val = 1)[0]
    return psnr_value


def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


img = np.zeros((1, 384, 384, 1)).astype('double')
train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ts_df['images'][
    193], color_mode = "grayscale", target_size = (384, 512))
train_img = image.img_to_array(train_igg)
train_img /= 255.0
img[0, :, :, :] = train_img[:, 64:448]
img = np.array(img)

model = load_model('checkpoint_loss6', custom_objects = {'PSNR': PSNR, 'SSIMLoss': SSIMLoss})
fe_model = Model(inputs = model.inputs, outputs = model.layers[12].output)  # 4, ..., 24

activation = fe_model(img)

plt.figure(figsize = (20, 20))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(activation[0, :, :, i])
plt.show()